# NLP with Bert for Sentiment Analysis

1.   List item
2.   List item



### Importing the libraries

In [1]:
!pip3 install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 39.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━

In [2]:
import os.path
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

In [3]:
from google.colab import files
uploaded = files.upload()

Saving subjects-questions.csv to subjects-questions.csv


In [5]:
# Create the data set

import pandas as pd
df = pd.read_csv('./subjects-questions.csv', header=None, usecols=[0, 1], names=['question', 'category'])
df = df[df['category'].isin(['Physics', 'Chemistry', 'Maths', 'Biology'])]
questions = df['question']

In [6]:
idxs = {"Biology": 0, "Chemistry": 1, "Maths": 2, "Physics": 3}

In [7]:
# y = df['category']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(questions, y, test_size=0.2, random_state=1)

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [8]:
import os

def build_dir(data, top_dir = "train", col_name = 'Biology'):
    if not os.path.isdir('./DATA'):
      os.mkdir('./DATA')
    top_dir = './DATA/' + top_dir
    if not os.path.isdir(top_dir):
        os.mkdir(top_dir)
    path = top_dir + "/" + col_name
    if not os.path.isdir(path):
        os.mkdir(path)

    for index, row in data.iterrows():
    # print(row['c1'], row['c2'])
        question = row['question']
        file_path = path + "/" + str(index) + "_" + col_name
        if not os.path.exists(file_path):
            with open(file_path, 'w+', encoding="utf-8") as writefile:
#                 print(question)
                writefile.write(question)
        else:
          print("Warning!")


def get_data_and_build_dir(col_name = 'Biology'):  # father_dir = "training", 
    # if father_dir == "training":
    train_col = train[train['category'] == col_name]
    test_col = test[test['category'] == col_name]
    # elif father_dir == "test":
    #   df_col = df[df['category'] == col_name]
    build_dir(train_col, "train", col_name)
    build_dir(test_col, "test", col_name)

    
for key in idxs.keys():
    print(key)
    get_data_and_build_dir(key)

print("Done!")

Biology
Chemistry
Maths
Physics
Done!


In [9]:
# from google.colab import drive
# drive.mount('/content/drive/')
# __dir__ = '/content/drive/My Drive/Colab Notebooks/NLP BERT 4 Text MultiClass Text Classification'
# import os
# os.chdir("/content/drive/")
# !ls

## Part 1: Data Preprocessing

### Loading the IMDB dataset

In [10]:
# dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
#                                   origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
#                                   extract=True)
# IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [11]:
# print(os.path.dirname(dataset))
# print(IMDB_DATADIR)

### Creating the training and test sets

In [12]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(datadir='./DATA',
                                                                       classes=list(idxs.keys()),
                                                                       maxlen=500,
                                                                       train_test_names=['train','test'],
                                                                       preprocess_mode='bert')

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## Part 2: Building the BERT model

In [13]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


## Part 3: Training the BERT model

In [14]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [15]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
16333/16333 [==============================] - 14164s 866ms/step - loss: 0.2155 - accuracy: 0.9226 - val_loss: 0.1250 - val_accuracy: 0.9541


In [16]:
# learner.fit_onecycle(lr=2e-5,
#                      epochs=2)

In [26]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [27]:
print(predictor.predict([""]))  # Let's see what it predicts

1/1 [==============================] - 7s 7s/step
['Chemistry']


In [28]:
print(predictor.predict(["isosceles traingle"]))  # Should be Math

1/1 [==============================] - 0s 99ms/step
['Maths']


In [32]:
print(predictor.predict(["the laws of motion and universal gravitation", "intra cell mitochondria", "the periodic table of elements"]))
# should be Physics, Biology, Chemistry

1/1 [==============================] - 0s 239ms/step
['Physics', 'Biology', 'Chemistry']


In [33]:
predictor.save("BERT_Multi_Class_Predictor")

In [34]:
from google.colab import files
files.download('BERT_Multi_Class_Predictor')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>